In [1]:
import os
import pandas as pd
import json
import re

In [2]:
class trialContextManager:
    def __enter__(self): pass
    def __exit__(self, *args): return True
trial = trialContextManager()

In [3]:
def read_json(file):
    with open(file, 'rb') as f:
        d = json.load(f)
    return(d)

In [66]:
def remove_header(d):
    # change to fit all formates
    for i in d:
        paragraphs = d[i].split('\n\n')

        header, jaargang, page_number, mnm_in_string = [False] * 4
        with trial: header = len(paragraphs[0]) < 200 
        with trial: jaargang = bool(re.search("\d{4}, jaargang \d{2}, nr. \d{1}", paragraphs))
        with trial: page_number = paragraphs[0][0].isdigit() | paragraphs[0][-1].isdigit()
        with trial: mnm_in_string = "Mens & Maatschappij" in paragraphs[0]
        
        if header & page_number | mnm_in_string | jaargang:
            paragraphs.pop(0)
        
        text = '\n\n'.join(paragraphs)\
            .replace('\S-\n', '')
        d[i] = text
    return(d)

In [60]:
def separate_body_and_footnotes(d):
    footnotes = []
    for i in d:
        paragraphs = d[i].split('\n\n')
        body = []
        for p in paragraphs: 
            if re.match(r'^\d+\)', p):
                footnotes.append(p)
            else:
                body.append(p)
        text = '\n\n'.join(body)
        d[i] = text
    return(d, footnotes)

In [61]:
def combine_text(d):
    text = ''.join(d.values()).replace('-\s?\n', '').replace('-\n', '')
    return(text)

In [62]:
def write_clean_text(d,f, dir):
    # save clean text
    outfile = os.path.join(dir, 'clean_text.txt')
    with open(outfile, mode='w') as fd:
        fd.write(d)
    
    # write footnotes if available
    if f:
        f_file = os.path.join(dir, 'footnotes.json')
        with open(f_file, 'w') as file:
            json.dump(f, file)

In [67]:
def remove_pagenumber(d):
    p_hold = []
    for paragraph in d.split('\n\n'):
        lines = paragraph.split('\n')
        hold = []
        for line in lines:
            if re.match('^[0-9]*$',line):
                pass
            else:
                hold.append(line)
            text = '\n'.join(hold)
        p_hold.append(text)
    d = '\n\n'.join(p_hold)
    return(d)

In [73]:
def main(dir, file):
    d = read_json(file)
    d = remove_header(d)
    d, f = separate_body_and_footnotes(d)
    d = combine_text(d)
    d = remove_pagenumber(d)
    write_clean_text(d,f, dir)

In [74]:
issues = os.listdir('database')
for issue in issues:
    articles = os.listdir(os.path.join('database',issue))
    for article in articles:
        dir = os.path.join('database', issue, article)
        file = os.path.join(dir, 'text.json')
        main(dir, file)



NameError: name 'f' is not defined